### Model SVM

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn import metrics

df=pd.read_csv('pre_content.csv')
# df=pd.read_csv('pre_title_steaming.csv')
df.head(10)

,content,sentimen,url_berita
0,presiden joko widodo memimpin upacara peringat...,0,https://20.detik.com/detikupdate/20231001-2310...
1,kpk mengungkap oknum menghilangkan barang bukt...,0,https://20.detik.com/detikupdate/20231001-2310...
2,remaja palestina umur 14 tahun ditembak mati p...,0,https://20.detik.com/detikupdate/20230831-2308...
3,presiden joko widodo memimpin upacara peringat...,0,https://20.detik.com/detikupdate/20231001-2310...
4,20detik merangkum kejadian heboh menarik sepek...,0,https://20.detik.com/detikupdate/20231001-2310...
5,sekjen pdip hasto kristiyanto berbicara terkai...,0,https://20.detik.com/detikupdate/20231001-2310...
6,sekjen pdip hasto kristiyanto membenarkan ganj...,0,https://20.detik.com/detikupdate/20231001-2310...
7,menko polhukam mahfud md buka suara temuan 12 ...,0,https://20.detik.com/detikupdate/20231001-2310...
8,saksi sidang korupsi bt 4g kominfo menyebut me...,0,https://20.detik.com/detikupdate/20231001-2310...
9,menko polhukam mahfud md buka suara oknum menc...,0,https://20.detik.com/detikupdate/20231001-2310...


In [2]:
def map_sentiment(sentiment):
    if sentiment == 1:
        return 'positif'
    elif sentiment == 0:
        return 'netral'
    else:
        return 'negatif'
df['sentimen'] = df['sentimen'].map(map_sentiment)


In [3]:
X = df['content']
y = df['sentimen']

In [4]:
def prepare_datasets(corpus, labels, test_data_proportion=0.3):
    train_X, test_X, train_Y, test_Y = train_test_split(corpus, labels,
    test_size=0.33,random_state=42)
    return train_X, test_X, train_Y, test_Y

norm_train_corpus, norm_test_corpus, train_labels, test_labels = prepare_datasets(X,y,test_data_proportion=0.3)

In [5]:
#Fungsi untuk mengekstraksi feature menggunakan TF-IDF Model
def tfidf_transformer(bow_matrix):
    transformer = TfidfTransformer(norm='l2',
                                    smooth_idf=True,
                                    use_idf=True)
    tfidf_matrix = transformer.fit_transform(bow_matrix)
    return transformer, tfidf_matrix
def tfidf_extractor(corpus, ngram_range=(1,2)):
    vectorizer = TfidfVectorizer(min_df=1,
                                norm='l2',
                                smooth_idf=True,
                                use_idf=True,
                                ngram_range=ngram_range)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

In [6]:
#mengunakan model TF-IDF untuk mengekstraksi feature
tfidf_vectorizer, tfidf_train_features = tfidf_extractor(norm_train_corpus)
tfidf_test_features = tfidf_vectorizer.transform(norm_test_corpus)

In [7]:
from sklearn.svm import SVC

In [8]:
# Create SVM model
svm_model = SVC(kernel='linear', random_state=42)
# Train SVM model
svm_model.fit(tfidf_train_features, train_labels)

SVC(kernel='linear', random_state=42)

In [9]:
# Predict
svm_pred = svm_model.predict(tfidf_test_features)

# Accuracy
svm_accuracy = accuracy_score(test_labels, svm_pred)
print("Accuracy:", svm_accuracy)

Accuracy: 0.6268686868686869


In [10]:
print(metrics.classification_report(test_labels, svm_pred))


              precision    recall  f1-score   support

     negatif       0.63      0.62      0.62      1646
      netral       0.61      0.61      0.61      1670
     positif       0.64      0.66      0.65      1634

    accuracy                           0.63      4950
   macro avg       0.63      0.63      0.63      4950
weighted avg       0.63      0.63      0.63      4950



In [11]:
test = ('Jeka Saragih On Fire Menatap Debut di UFC')
line = tfidf_vectorizer.transform([test])
line.shape
test_result = svm_model.predict(line)
print("Hasil Prediksi:", test_result) 


Hasil Prediksi: ['netral']
